In [ ]:
# Cell1:  Shared Cache Bootstrap
import os, pathlib, torch
import sys
from datetime import datetime

# Shared cache configuration (複製到每本 notebook)
AI_CACHE_ROOT = os.getenv("AI_CACHE_ROOT", "../ai_warehouse/cache")

for k, v in {
    "HF_HOME": f"{AI_CACHE_ROOT}/hf",
    "TRANSFORMERS_CACHE": f"{AI_CACHE_ROOT}/hf/transformers",
    "HF_DATASETS_CACHE": f"{AI_CACHE_ROOT}/hf/datasets",
    "HUGGINGFACE_HUB_CACHE": f"{AI_CACHE_ROOT}/hf/hub",
    "TORCH_HOME": f"{AI_CACHE_ROOT}/torch",
}.items():
    os.environ[k] = v
    pathlib.Path(v).mkdir(parents=True, exist_ok=True)
print("[Cache]", AI_CACHE_ROOT, "| GPU:", torch.cuda.is_available())

In [ ]:
# Cell 2: Dependencies & Imports
import gradio as gr
import json
import time
from typing import List, Tuple, Dict, Any

In [ ]:
# Cell 3: Theme System Base
DARK_THEME_CSS = """
/* Dark theme variables */
:root {
    --bg-primary: #1a1a1a;
    --bg-secondary: #2d2d2d;
    --bg-tertiary: #3a3a3a;
    --text-primary: #ffffff;
    --text-secondary: #cccccc;
    --border-color: #404040;
    --accent-color: #4a9eff;
    --hover-color: #4a4a4a;
    --success-color: #28a745;
    --warning-color: #ffc107;
    --error-color: #dc3545;
}

.dark-theme {
    background-color: var(--bg-primary) !important;
    color: var(--text-primary) !important;
}

.dark-theme .gr-button {
    background-color: var(--bg-secondary) !important;
    color: var(--text-primary) !important;
    border: 1px solid var(--border-color) !important;
}

.dark-theme .gr-button:hover {
    background-color: var(--hover-color) !important;
}

.dark-theme .gr-textbox {
    background-color: var(--bg-secondary) !important;
    color: var(--text-primary) !important;
    border: 1px solid var(--border-color) !important;
}

.dark-theme .gr-chatbot {
    background-color: var(--bg-secondary) !important;
    border: 1px solid var(--border-color) !important;
}

.dark-theme .gr-chatbot .message {
    background-color: var(--bg-tertiary) !important;
    color: var(--text-primary) !important;
}

/* Scroll improvements */
.chat-container {
    max-height: 500px;
    overflow-y: auto;
    scroll-behavior: smooth;
}

.auto-scroll {
    scroll-behavior: smooth;
}

/* Shortcuts hint */
.shortcuts-hint {
    position: fixed;
    bottom: 10px;
    right: 10px;
    background-color: var(--bg-tertiary);
    color: var(--text-secondary);
    padding: 8px 12px;
    border-radius: 6px;
    font-size: 12px;
    opacity: 0.7;
    z-index: 1000;
}
"""

LIGHT_THEME_CSS = """
/* Light theme variables */
:root {
    --bg-primary: #ffffff;
    --bg-secondary: #f8f9fa;
    --bg-tertiary: #e9ecef;
    --text-primary: #212529;
    --text-secondary: #6c757d;
    --border-color: #dee2e6;
    --accent-color: #007bff;
    --hover-color: #e2e6ea;
    --success-color: #28a745;
    --warning-color: #ffc107;
    --error-color: #dc3545;
}

.light-theme {
    background-color: var(--bg-primary) !important;
    color: var(--text-primary) !important;
}

.light-theme .gr-button {
    background-color: var(--bg-secondary) !important;
    color: var(--text-primary) !important;
    border: 1px solid var(--border-color) !important;
}

.light-theme .gr-button:hover {
    background-color: var(--hover-color) !important;
}
"""

In [ ]:
# Cell 4: Keyboard Shortcuts Handler
SHORTCUTS_JS = """
function setupKeyboardShortcuts() {
    let shortcuts = {
        'ctrl+enter': 'send_message',
        'ctrl+l': 'clear_chat',
        'ctrl+1': 'mode_chat',
        'ctrl+2': 'mode_rag',
        'ctrl+3': 'mode_agents',
        'ctrl+4': 'mode_game',
        'ctrl+d': 'toggle_theme',
        'escape': 'cancel_generation'
    };

    document.addEventListener('keydown', function(e) {
        let key = '';
        if (e.ctrlKey) key += 'ctrl+';
        if (e.shiftKey) key += 'shift+';
        if (e.altKey) key += 'alt+';

        if (e.key === 'Enter') key += 'enter';
        else if (e.key === 'Escape') key += 'escape';
        else if (e.key.length === 1) key += e.key.toLowerCase();
        else key += e.key.toLowerCase();

        if (shortcuts[key]) {
            e.preventDefault();
            handleShortcut(shortcuts[key]);
        }
    });
}

function handleShortcut(action) {
    console.log('Shortcut triggered:', action);

    switch(action) {
        case 'send_message':
            // Trigger send button if input has content
            let textInput = document.querySelector('textarea');
            let sendBtn = document.querySelector('button[variant="primary"]');
            if (textInput && textInput.value.trim() && sendBtn) {
                sendBtn.click();
            }
            break;

        case 'clear_chat':
            // Find and click clear button
            let clearBtn = document.querySelector('button:contains("清空")');
            if (clearBtn) clearBtn.click();
            break;

        case 'toggle_theme':
            // Find theme toggle button
            let themeBtn = document.querySelector('#theme-toggle');
            if (themeBtn) themeBtn.click();
            break;

        case 'mode_chat':
        case 'mode_rag':
        case 'mode_agents':
        case 'mode_game':
            // Switch modes via radio buttons
            let modeNum = action.split('_')[1];
            let modeRadio = document.querySelector(`input[value="${modeNum}"]`);
            if (modeRadio) modeRadio.click();
            break;

        case 'cancel_generation':
            // Find stop button during generation
            let stopBtn = document.querySelector('button:contains("停止")');
            if (stopBtn) stopBtn.click();
            break;
    }
}

function autoScrollToBottom() {
    let chatContainer = document.querySelector('.gr-chatbot');
    if (chatContainer) {
        chatContainer.scrollTop = chatContainer.scrollHeight;
    }
}

// Initialize shortcuts when DOM loads
if (document.readyState === 'loading') {
    document.addEventListener('DOMContentLoaded', setupKeyboardShortcuts);
} else {
    setupKeyboardShortcuts();
}
"""

In [ ]:
# Cell 5: Auto-scroll & Chat UX
class ThemeManager:
    def __init__(self):
        self.current_theme = "light"
        self.themes = {"light": LIGHT_THEME_CSS, "dark": DARK_THEME_CSS}

    def get_current_css(self) -> str:
        """Get CSS for current theme"""
        base_css = """
        /* Base responsive design */
        .gradio-container {
            max-width: 1200px !important;
            margin: 0 auto !important;
        }

        /* Improved chat styling */
        .gr-chatbot {
            height: 400px !important;
            overflow-y: auto !important;
        }

        /* Button improvements */
        .gr-button {
            transition: all 0.2s ease !important;
        }

        .gr-button:hover {
            transform: translateY(-1px) !important;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1) !important;
        }
        """

        return base_css + self.themes[self.current_theme]

    def toggle_theme(self) -> Tuple[str, str]:
        """Toggle between light and dark theme"""
        self.current_theme = "dark" if self.current_theme == "light" else "light"
        theme_name = "🌙 深色" if self.current_theme == "dark" else "☀️ 淺色"
        return self.get_current_css(), theme_name

In [ ]:
# Cell 6: Complete Themed UI
def create_themed_chat_interface():
    """Create complete themed chat interface with shortcuts"""
    theme_manager = ThemeManager()

    # Mock LLM response for demo
    def mock_chat_response(
        message: str, history: List[List[str]], mode: str
    ) -> Tuple[str, List[List[str]]]:
        if not message.strip():
            return "", history

        # Simulate processing delay
        time.sleep(0.5)

        responses = {
            "chat": f"[Chat模式] 收到訊息：{message}",
            "rag": f"[RAG模式] 基於知識庫回答：{message}",
            "agents": f"[Agents模式] 多代理協作處理：{message}",
            "game": f"[Game模式] 遊戲互動：{message}",
        }

        response = responses.get(mode, f"未知模式回應：{message}")
        history.append([message, response])
        return "", history

    def clear_chat() -> List[List[str]]:
        """Clear chat history"""
        return []

    def toggle_theme_handler(current_css: str, current_name: str) -> Tuple[str, str]:
        """Handle theme toggle"""
        return theme_manager.toggle_theme()

    # Create interface
    with gr.Blocks(
        css=theme_manager.get_current_css(),
        title="Themed Chat Interface",
        head=f"<script>{SHORTCUTS_JS}</script>",
    ) as demo:

        # State variables
        current_css = gr.State(theme_manager.get_current_css())

        # Header with theme controls
        with gr.Row():
            gr.Markdown("# 🎨 主題化聊天介面")
            theme_toggle = gr.Button(
                "☀️ 淺色", elem_id="theme-toggle", variant="secondary", size="sm"
            )

        # Shortcuts hint
        gr.HTML(
            """
        <div class="shortcuts-hint">
            快捷鍵：Ctrl+Enter(發送) | Ctrl+L(清空) | Ctrl+D(主題) | Ctrl+1-4(模式)
        </div>
        """
        )

        # Main interface
        with gr.Row():
            # Left: Chat
            with gr.Column(scale=3):
                chatbot = gr.Chatbot(
                    value=[], height=400, elem_classes=["chat-container"]
                )

                with gr.Row():
                    msg_input = gr.Textbox(
                        placeholder="輸入訊息... (Ctrl+Enter 發送)",
                        scale=4,
                        show_label=False,
                    )
                    send_btn = gr.Button("發送", variant="primary", scale=1)

                with gr.Row():
                    clear_btn = gr.Button("清空", variant="secondary")
                    stop_btn = gr.Button("停止", variant="stop", visible=False)

            # Right: Controls
            with gr.Column(scale=1):
                gr.Markdown("### 模式選擇")
                mode = gr.Radio(
                    choices=["chat", "rag", "agents", "game"],
                    value="chat",
                    label="運行模式",
                )

                gr.Markdown("### 主題設定")
                theme_info = gr.Markdown("當前：淺色主題")

                gr.Markdown("### 狀態")
                status = gr.Markdown("🟢 就緒")

        # Event handlers
        def send_message(message, history, current_mode):
            if message.strip():
                # Show processing status
                return mock_chat_response(message, history, current_mode)
            return "", history

        # Button events
        send_btn.click(
            send_message,
            inputs=[msg_input, chatbot, mode],
            outputs=[msg_input, chatbot],
        )

        msg_input.submit(
            send_message,
            inputs=[msg_input, chatbot, mode],
            outputs=[msg_input, chatbot],
        )

        clear_btn.click(clear_chat, outputs=[chatbot])

        # Theme toggle
        theme_toggle.click(
            toggle_theme_handler,
            inputs=[current_css, theme_toggle],
            outputs=[current_css, theme_toggle],
        ).then(
            lambda css: css,
            inputs=[current_css],
            outputs=[],
            js="(css) => { document.querySelector('style').textContent = css; }",
        )

        # Auto-scroll after each message
        chatbot.change(None, js="() => { setTimeout(autoScrollToBottom, 100); }")

    return demo

In [ ]:
# Cell 7: Smoke Test
def smoke_test_themed_interface():
    """Quick test of theming and shortcuts functionality"""
    print("🧪 Smoke Test: Themed Interface")

    # Test theme manager
    theme_mgr = ThemeManager()
    print(f"✅ Default theme: {theme_mgr.current_theme}")

    # Test theme toggle
    css, name = theme_mgr.toggle_theme()
    print(f"✅ Toggled to: {theme_mgr.current_theme} ({name})")

    # Test CSS generation
    css_length = len(theme_mgr.get_current_css())
    print(f"✅ Generated CSS: {css_length} characters")

    # Test shortcuts JS
    js_length = len(SHORTCUTS_JS)
    print(f"✅ Shortcuts JS: {js_length} characters")

    print("\n🎯 Ready to launch themed interface!")
    return True


# Run smoke test
smoke_test_themed_interface()

# Launch interface
if __name__ == "__main__":
    demo = create_themed_chat_interface()
    demo.launch(server_name="0.0.0.0", server_port=7860, share=False, debug=True)

In [ ]:
# CSS 變數系統
THEME_VARIABLES = {
    "dark": {
        "--bg-primary": "#1a1a1a",
        "--text-primary": "#ffffff",
        "--accent-color": "#4a9eff",
    },
    "light": {
        "--bg-primary": "#ffffff",
        "--text-primary": "#212529",
        "--accent-color": "#007bff",
    },
}

In [ ]:
# 快捷鍵映射
SHORTCUTS_MAP = {
    "ctrl+enter": "send_message",  # 發送訊息
    "ctrl+l": "clear_chat",  # 清空聊天
    "ctrl+1-4": "mode_switch",  # 模式切換
    "ctrl+d": "toggle_theme",  # 主題切換
    "escape": "cancel_generation",  # 取消生成
}

In [ ]:
# 快速驗證主題與快捷鍵功能
def quick_smoke_test():
    # 1. 測試主題管理器
    theme_mgr = ThemeManager()
    assert theme_mgr.current_theme == "light"

    # 2. 測試主題切換
    css, name = theme_mgr.toggle_theme()
    assert theme_mgr.current_theme == "dark"
    assert "1a1a1a" in css  # 深色背景

    # 3. 測試 CSS 生成
    css_output = theme_mgr.get_current_css()
    assert len(css_output) > 500  # 合理的 CSS 長度

    # 4. 測試 JavaScript 載入
    assert "setupKeyboardShortcuts" in SHORTCUTS_JS
    assert "ctrl+enter" in SHORTCUTS_JS

    print("✅ 所有煙霧測試通過！")


quick_smoke_test()